# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770645526290                   -0.53    8.0
  2   -2.772130011997       -2.83       -1.28    1.0
  3   -2.772170543422       -4.39       -2.95    2.0
  4   -2.772170720638       -6.75       -3.60    3.0
  5   -2.772170722935       -8.64       -4.32    1.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770791895973                   -0.52    8.0
  2   -2.772056697859       -2.90       -1.32    1.0
  3   -2.772083038621       -4.58       -2.46    2.0
  4   -2.772083417338       -6.42       -4.14    2.0
  5   -2.772083417784       -9.35       -4.96    2.0


1.7735518430582442

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770668546622                   -0.53    8.0
  2   -2.772048444450       -2.86       -1.30    1.0
  3   -2.772083368172       -4.46       -2.75    2.0
  4   -2.772083416718       -7.31       -4.00    2.0
  5   -2.772083417785       -8.97       -4.63    2.0

Polarizability via ForwardDiff:       1.772557455357172
Polarizability via finite difference: 1.7735518430582442
